In [17]:
#Import libraries
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import norm
from scipy import stats
import scipy as sp
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")
plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [18]:
#read in data
df = pd.read_csv('listings.csv', index_col=0)

## Cleaning Data Columns
This notebook is removing the majority of text based columns to make the sheet more usuable

In [19]:
df.head()

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,https://www.airbnb.com/rooms/958,2.019100e+13,10/14/19,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,NaN,NaN,https://a0.muscache.com/im/pictures/b7c2a199-4...,NaN,1169,https://www.airbnb.com/users/show/1169,Holly,7/31/08,"San Francisco, California, United States",We are a family with 2 boys born in 2009 and 2...,within a day,100%,NaN,t,https://a0.muscache.com/im/pictures/efdad96a-3...,https://a0.muscache.com/im/pictures/efdad96a-3...,Duboce Triangle,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"San Francisco, CA, United States",Duboce Triangle,Western Addition,NaN,San Francisco,CA,94117,San Francisco,"San Francisco, CA",US,United States,37.76931,-122.43386,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,1,1,30,30,1.0,30.0,2 months ago,t,3,8,16,85,10/14/19,217,52,7/23/09,9/21/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1,0,0,1.74
3850,https://www.airbnb.com/rooms/3850,2.019100e+13,10/14/19,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,none,"This is a quiet, safe neighborhood on a substa...",House Rule footnotes: 1.\tI don’t allow check ...,Public transit service to my house is outstand...,"During the hours of 7:00 to 22:00, your access...","I live at the house in my own bedroom, and I w...",Airbnb has no default house rules that hosts c...,NaN,NaN,https://a0.muscache.com/im/pictures/b6e97d2a-1...,NaN,4921,https://www.airbnb.com/users/show/4921,Kevin,12/8/08,"San Franci

In [20]:
df.dtypes

listing_url                                      object
scrape_id                                       float64
last_scraped                                     object
name                                             object
summary                                          object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 105, dtype: object

In [21]:
df.shape

(8111, 105)

In [22]:
#Dropping text columns
df.drop(['listing_url','scrape_id','last_scraped','name','summary','space','description'],axis=1,inplace = True)
df.drop(['neighborhood_overview','transit','access','interaction','house_rules','thumbnail_url','medium_url'],axis=1,inplace = True)
df.drop(['host_acceptance_rate','host_response_time','host_location','host_url','host_id','xl_picture_url','picture_url'],axis=1,inplace = True)


In [23]:
df.head()

,experiences_offered,notes,host_name,host_since,host_about,host_response_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,none,Due to the fact that we have children and a do...,Holly,7/31/08,We are a family with 2 boys born in 2009 and 2...,100%,t,https://a0.muscache.com/im/pictures/efdad96a-3...,https://a0.muscache.com/im/pictures/efdad96a-3...,Duboce Triangle,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"San Francisco, CA, United States",Duboce Triangle,Western Addition,NaN,San Francisco,CA,94117,San Francisco,"San Francisco, CA",US,United States,37.76931,-122.43386,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,1,1,30,30,1.0,30.0,2 months ago,t,3,8,16,85,10/14/19,217,52,7/23/09,9/21/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1,0,0,1.74
3850,none,House Rule footnotes: 1.\tI don’t allow check ...,Kevin,12/8/08,I am a retired software developer now raising ...,100%,t,https://a0.muscache.com/im/pictures/97d29791-d...,https://a0.muscache.com/im/pictures/97d29791-d...,Inner Sunset,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"San Francisco, CA, United States",Inner Sunset,Inner Sunset,NaN,San Francisco,CA,94131,San Francisco,"San Francisco, CA",US,United States,37.75402,-122.45805,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{Internet,Wifi,Kitchen,Breakfast,""Free street ...",NaN,$99.00,NaN,NaN,$0.00,$10.00,2,$20.00,1,5,1,1,5,5,1.0,5.0,3 days ago,t,5,32,62,62,10/14/19,160,36,7/16/09,9/30/19,94.0,10.0,10.0,10.0,10.0,10.0,10.0,t,STR-0000346,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,3,0,3,0,1.28
5858,none,All the furniture in the house was handmade so...,Philip And Tania,3/2/09,Philip: English transplant to the Bay Area and...,80%,f,https://a0.muscache.com/im/users/8904/profile_...,https://a0.muscache.com/im/users/8904/profile_...,Bernal Heights,2.0,2.0,"['email', 'phone', 'reviews', 'kba', 'work_ema...",t,t,"San Francisco, CA, United States",Bernal Heights,Bernal Heights,NaN,San Francisco,CA,94110,San Francisco,"San Francisco, CA",US,United States,37.74511,-122.42102,t,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,"{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",NaN,$235.00,"$1,600.00","$5,500.00",NaN,$100.00,2,$0.00,30,60,30,30,60,60,30.0,60.0,3 weeks ago,t,0,0,0,0,10/14/19,111,0,5/3/09,8/6/17,98.0,10.0,10.0,10.0,10.0,10.0,9.0,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_gra

In [24]:
df.shape

(8111, 84)

In [25]:
#Dropping text columns
df.drop(['notes','host_about','host_thumbnail_url','host_picture_url','host_listings_count','host_verifications','host_neighbourhood'],axis=1,inplace = True)
df.drop(['is_location_exact','smart_location','neighbourhood_group_cleansed','neighbourhood','street','host_identity_verified','host_has_profile_pic'],axis=1,inplace = True)
df.drop(['bed_type','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm'],axis=1,inplace = True)


In [26]:
df.head()

,experiences_offered,host_name,host_since,host_response_rate,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,city,state,zipcode,market,country_code,country,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,none,Holly,7/31/08,100%,t,1.0,Western Addition,San Francisco,CA,94117,San Francisco,US,United States,37.76931,-122.43386,Apartment,Entire home/apt,3,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets liv...",NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,2 months ago,t,3,8,16,85,10/14/19,217,52,7/23/09,9/21/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1,0,0,1.74
3850,none,Kevin,12/8/08,100%,t,2.0,Inner Sunset,San Francisco,CA,94131,San Francisco,US,United States,37.75402,-122.45805,House,Private room,2,1.0,1.0,1.0,"{Internet,Wifi,Kitchen,Breakfast,""Free street ...",NaN,$99.00,NaN,NaN,$0.00,$10.00,2,$20.00,1,5,3 days ago,t,5,32,62,62,10/14/19,160,36,7/16/09,9/30/19,94.0,10.0,10.0,10.0,10.0,10.0,10.0,t,STR-0000346,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,3,0,3,0,1.28
5858,none,Philip And Tania,3/2/09,80%,f,2.0,Bernal Heights,San Francisco,CA,94110,San Francisco,US,United States,37.74511,-122.42102,Apartment,Entire home/apt,5,1.0,2.0,3.0,"{Internet,Wifi,Kitchen,Heating,""Family/kid fri...",NaN,$235.00,"$1,600.00","$5,500.00",NaN,$100.00,2,$0.00,30,60,3 weeks ago,t,0,0,0,0,10/14/19,111,0,5/3/09,8/6/17,98.0,10.0,10.0,10.0,10.0,10.0,9.0,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.87
7918,none,Aaron,6/17/09,86%,t,10.0,Haight Ashbury,San Francisco,CA,94117,San Francisco,US,United States,37.76669,-122.45250,Apartment,Private room,2,4.0,1.0,1.0,"{TV,Internet,Wifi,Kitchen,""Free street parking...",NaN,$65.00,$485.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,60,3 months ago,t,30,60,90,365,10/14/19,18,1,8/31/09,9/10/19,86.0,8.0,8.0,9.0,9.0,9.0,8.0,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.15
8142,none,Aaron,6/17/09,86%,t,10.0,Haight Ashbury,San Francisco,CA,94117,San Francisco,US,United States,37.76487,-122.45183,Apartment,Private room,2,4.0,1.0,1.0,"{TV,Internet,Wifi,Kitchen,""Free street parking...",NaN,$65.00,$490.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,90,3 months ago,t,30,60,90,365,10/14/19,8,0,9/8/14,9/12/18,93.0,9.0,9.0,10.0,10.0,9.0,9.0,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.13


In [27]:
df.shape

(8111, 63)

In [28]:
#Dropping text columns
df.drop(['amenities','calendar_updated','has_availability','availability_30','availability_60','availability_90','availability_365'],axis=1,inplace = True)
df.drop(['review_scores_checkin','review_scores_cleanliness','review_scores_accuracy','last_review','first_review','number_of_reviews_ltm','calendar_last_scraped'],axis=1,inplace = True)
df.drop(['review_scores_communication','review_scores_location','review_scores_value','requires_license','license','jurisdiction_names','cancellation_policy'],axis=1,inplace = True)
df.drop(['is_business_travel_ready','require_guest_phone_verification','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms'],axis=1,inplace = True)
df.drop(['host_name','host_since','market','country','country_code','state','require_guest_profile_picture'],axis=1,inplace = True)


In [29]:
df.head()

,experiences_offered,host_response_rate,host_is_superhost,host_total_listings_count,neighbourhood_cleansed,city,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
958,none,100%,t,1.0,Western Addition,San Francisco,94117,37.76931,-122.43386,Apartment,Entire home/apt,3,1.0,1.0,2.0,NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,217,97.0,f,1.74
3850,none,100%,t,2.0,Inner Sunset,San Francisco,94131,37.75402,-122.45805,House,Private room,2,1.0,1.0,1.0,NaN,$99.00,NaN,NaN,$0.00,$10.00,2,$20.00,1,5,160,94.0,f,1.28
5858,none,80%,f,2.0,Bernal Heights,San Francisco,94110,37.74511,-122.42102,Apartment,Entire home/apt,5,1.0,2.0,3.0,NaN,$235.00,"$1,600.00","$5,500.00",NaN,$100.00,2,$0.00,30,60,111,98.0,f,0.87
7918,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76669,-122.45250,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$485.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,60,18,86.0,f,0.15
8142,none,86%,t,10.0,Haight Ashbury,San Francisco,94117,37.76487,-122.45183,Apartment,Private room,2,4.0,1.0,1.0,NaN,$65.00,$490.00,"$1,685.00",$200.00,$50.00,1,$12.00,32,90,8,93.0,f,0.13


In [30]:
df.shape

(8111, 29)

In [31]:
df.to_csv('listingsclean.csv')